# Method that allows you to draw bounding boxes on the objects in all the images within a directory




In [1]:
import os
import cv2
import json

In [2]:
class ImageAreaSelector:
    def __init__(self):
        self.refPt = []
        self.areas = []
        self.cropping = False

    def click_and_crop(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.refPt = [(x, y)]
            self.cropping = True
            self.image_with_rect = self.image.copy()

        elif event == cv2.EVENT_MOUSEMOVE:
            if self.cropping:
                temp_image = self.image_with_rect.copy()
                cv2.rectangle(temp_image, self.refPt[0], (x, y), (0, 255, 0), 2)
                cv2.imshow("image", temp_image)

        elif event == cv2.EVENT_LBUTTONUP:
            self.refPt.append((x, y))
            self.cropping = False
            cv2.rectangle(self.image_with_rect, self.refPt[0], (x, y), (0, 255, 0), 2)
            cv2.imshow("image", self.image_with_rect)
            self.areas.append(self.refPt)

    def select_areas(self, image):
        self.image = image.copy()
        self.image_with_rect = image.copy()
        cv2.namedWindow("image")
        cv2.imshow("image", self.image) 
        cv2.setMouseCallback("image", self.click_and_crop)

    def remove_last_area(self):
        if self.areas:
            self.areas.pop()
            self.image_with_rect = self.image.copy()
            for area in self.areas:
                cv2.rectangle(self.image_with_rect, area[0], area[1], (0, 255, 0), 2)
            cv2.imshow("image", self.image_with_rect)

## Press space to move to the next image and "d" to delete last bounding box


After passing through all the images, save the data in the desired format for Faster RCNN in a .json file.

In [3]:
image_folder = "./images/keys"
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
image_files = sorted(image_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
areas_dict = {}

annotations = []
categories = [
    {"id": 91, "name": "keys"},
    {"id": 92, "name": "pencil"},
    {"id": 93, "name": "sneakers"}
]

selector = ImageAreaSelector()

for idx, image_file in enumerate(image_files):
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)
    if image is None:
        print(f"No se pudo cargar la imagen {image_file}")
        continue

    selector.select_areas(image)
    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == ord(" "):  # If the space key is pressed, move to the next one
            break
        elif key == ord("d"):  # If "d" is pressed, delete last bounding box
            selector.remove_last_area()

    areas_dict[image_file] = selector.areas
    
    for area in areas_dict[image_file]:
        xmin, ymin = area[0]
        xmax, ymax = area[1]
        print ("xmin, ymin, xmax, ymax :", xmin, ymin, xmax, ymax)
        if "llaves" in image_file:
            category_id = 91
        elif "pencil" in image_file:
            category_id = 92
        elif "zapatilas" in image_file:
            category_id = 93
        else:
            category_id = 0  # Otra categoría por defecto
        annotations.append({"image_id": idx, "category_id": category_id, "bbox": [xmin, xmax, ymin, ymax]})
        print("annotations: ", annotations[-1])
        
    selector.areas = []

    cv2.destroyAllWindows()

json_data = {"annotations": annotations, "categories": categories, "images": [{"id": i, "file_name": image_files[i]} for i in range(len(image_files))]}

with open("annotations_perso.json", "w") as json_file:
    json.dump(json_data, json_file, indent=4)

print("Annotations saved to annotations_perso.json")

xmin, ymin, xmax, ymax : 21 12 150 93
annotations:  {'image_id': 0, 'category_id': 91, 'bbox': [21, 150, 12, 93]}
xmin, ymin, xmax, ymax : 4 67 156 4
annotations:  {'image_id': 1, 'category_id': 91, 'bbox': [4, 156, 67, 4]}
xmin, ymin, xmax, ymax : 1 2 137 139
annotations:  {'image_id': 2, 'category_id': 91, 'bbox': [1, 137, 2, 139]}
xmin, ymin, xmax, ymax : 4 6 155 95
annotations:  {'image_id': 3, 'category_id': 91, 'bbox': [4, 155, 6, 95]}
xmin, ymin, xmax, ymax : 8 98 152 14
annotations:  {'image_id': 4, 'category_id': 91, 'bbox': [8, 152, 98, 14]}
xmin, ymin, xmax, ymax : 5 112 159 22
annotations:  {'image_id': 5, 'category_id': 91, 'bbox': [5, 159, 112, 22]}
xmin, ymin, xmax, ymax : 8 139 134 -1
annotations:  {'image_id': 6, 'category_id': 91, 'bbox': [8, 134, 139, -1]}
xmin, ymin, xmax, ymax : 79 27 128 141
annotations:  {'image_id': 6, 'category_id': 91, 'bbox': [79, 128, 27, 141]}
xmin, ymin, xmax, ymax : 2 80 160 -1
annotations:  {'image_id': 7, 'category_id': 91, 'bbox': [2, 